In [1]:
import os
import cv2
import numpy as np
import pandas as pd
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import peak_signal_noise_ratio as psnr

In [3]:
#  PRD Function 
def calculate_prd(original, reconstructed):
    numerator = np.sum((original - reconstructed) ** 2)
    denominator = np.sum(original ** 2)
    return np.sqrt(numerator / denominator) * 100 if denominator != 0 else float('inf')

#  Main Metrics Function 
def calculate_metrics(original_path, compressed_path):
    original = cv2.imread(original_path, cv2.IMREAD_GRAYSCALE)
    compressed = cv2.imread(compressed_path, cv2.IMREAD_GRAYSCALE)

    if original is None or compressed is None:
        return None

    if original.shape != compressed.shape:
        print(f"Skipping {os.path.basename(original_path)}: size mismatch")
        return None

    ssim_val = ssim(original, compressed)
    psnr_val = psnr(original, compressed)
    prd_val = calculate_prd(original.astype(np.float64), compressed.astype(np.float64))

    # Compression ratio
    original_size = os.path.getsize(original_path)
    compressed_size = os.path.getsize(compressed_path)
    cr = original_size / compressed_size if compressed_size != 0 else float('inf')

    return {
        'Image Name': os.path.basename(original_path),
        'Compression Ratio': cr,
        'PSNR': psnr_val,
        'SSIM': ssim_val,
        'PRD': prd_val
    }

#  Batch Processing 
original_folder = 'org_ecg_10sec_resize'
compressed_folder = 'caearithmetic'

results = []

for filename in os.listdir(original_folder):
    if filename.lower().endswith(('.png', '.bmp', '.jpg')):
        orig_path = os.path.join(original_folder, filename)
        comp_path = os.path.join(compressed_folder, filename)

        if os.path.exists(comp_path):
            metrics = calculate_metrics(orig_path, comp_path)
            if metrics:
                results.append(metrics)
        else:
            print(f"Compressed version missing for {filename}")

# ---- Save to Excel ----
df = pd.DataFrame(results)
df.to_excel('caearithmetic.xlsx', index=False)
print("✅ Metrics saved to caearithmetic.xlsx")

✅ Metrics saved to caearithmetic.xlsx
